In [1]:
import dash
from dash import html, dcc
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output

# Load data
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
df = pd.read_csv(url)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='launch-site-dropdown',
        options=[{'label': site, 'value': site} for site in df['Launch Site'].unique()],
        value='All Sites',
        placeholder="Select a Launch Site",
        searchable=True
    ),
    html.Div(id='success-pie-chart-container')
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
@app.callback(
    Output(component_id='success-pie-chart-container', component_property='children'),
    Input(component_id='launch-site-dropdown', component_property='value')
)
def update_success_pie_chart(selected_site):
    if selected_site == 'All Sites':
        filtered_df = df
    else:
        filtered_df = df[df['Launch Site'] == selected_site]

    fig = px.pie(filtered_df, names='class', title=f'Launch Success Rates for {selected_site}')
    return dcc.Graph(figure=fig)
# Extend the layout in the app.layout section
app.layout = html.Div([
    dcc.Dropdown(
        id='launch-site-dropdown',
        options=[{'label': site, 'value': site} for site in df['Launch Site'].unique()],
        value='All Sites',
        placeholder="Select a Launch Site",
        searchable=True
    ),
    dcc.RangeSlider(
        id='payload-range-slider',
        min=df['Payload Mass (kg)'].min(),
        max=df['Payload Mass (kg)'].max(),
        step=1000,
        marks={i: str(i) for i in range(int(df['Payload Mass (kg)'].min()), int(df['Payload Mass (kg)'].max()), 1000)},
        value=[df['Payload Mass (kg)'].min(), df['Payload Mass (kg)'].max()]
    ),
    html.Div(id='success-pie-chart-container'),
    html.Div(id='success-payload-scatter-chart-container')
])
@app.callback(
    Output('success-payload-scatter-chart-container', 'children'),
    [Input('launch-site-dropdown', 'value'),
     Input('payload-range-slider', 'value')]
)
def update_payload_scatter_chart(selected_site, payload_range):
    filtered_df = df[(df['Payload Mass (kg)'] >= payload_range[0]) & (df['Payload Mass (kg)'] <= payload_range[1])]
    if selected_site != 'All Sites':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
    
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                     title='Success by Payload Mass and Booster Version')
    return dcc.Graph(figure=fig)
